In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings("ignore")
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
# hol = pd.read_csv('HOLLISTR_1_N101.csv')
# dam = pd.read_csv('HOLLISTR_1_N101_DAM.csv')

In [2]:
df = pd.read_csv('20220715_20220729_PRC_INTVL_LMP_RTM_20220731_05_17_54_v3.csv')
df.drop(df[df.LMP_TYPE == 'MGHG'].index, axis=0, inplace=True)

trash = ['NODE_ID_XML', 'NODE_ID', 'NODE', 'MARKET_RUN_ID', 'PNODE_RESMRID', 'GRP_TYPE', 'POS']
df.drop(trash, axis=1, inplace=True)
df.drop('XML_DATA_ITEM', axis=1, inplace=True)                      # Because it's 100% correlated with LMP_TYPE

date = ['INTERVALSTARTTIME_GMT', 'INTERVALENDTIME_GMT',	'OPR_DT', 'GROUP']

# date = ['INTERVALSTARTTIME_GMT', 'INTERVALENDTIME_GMT',	'OPR_DT']   # Because OPR_HR and OPR_INTERVAL contain it.
# df.drop(date, axis=1, inplace=True)

# 07:00 - 08:00    OPR_HR == 1
# 08:00 - 09:00    OPR_HR == 2
# 15:00 - 16:00    OPR_HR == 9
# 22:00 - 23:00    OPR_HR == 16

# 07:00 - 07:05    OPR_INTERVAL == 1

# df.XML_DATA_ITEM.unique() == ['LMP_CONG_PRC', 'LMP_ENE_PRC', 'LMP_GHG_PRC', 'LMP_LOSS_PRC', 'LMP_PRC']
# df.LMP_TYPE.unique() == ['MCC', 'MCE', 'MGHG', 'MCL', 'LMP']

In [3]:
le = LabelEncoder()
le.fit(['MCC', 'MCE', 'MCL', 'LMP'])   #  'MGHG' leads to VALUE == 0

df['lmp_trans'] = le.transform(df.LMP_TYPE)
df.drop('LMP_TYPE', axis=1, inplace=True)
df

,INTERVALSTARTTIME_GMT,INTERVALENDTIME_GMT,OPR_DT,OPR_HR,VALUE,OPR_INTERVAL,GROUP,lmp_trans
0,2022-07-15T15:00:00-00:00,2022-07-15T15:05:00-00:00,2022-07-15,9,20.05033,1,1,1
1,2022-07-15T15:05:00-00:00,2022-07-15T15:10:00-00:00,2022-07-15,9,25.44488,2,1,1
2,2022-07-15T15:10:00-00:00,2022-07-15T15:15:00-00:00,2022-07-15,9,27.20417,3,1,1
3,2022-07-15T15:15:00-00:00,2022-07-15T15:20:00-00:00,2022-07-15,9,19.84158,4,1,1
4,2022-07-15T15:20:00-00:00,2022-07-15T15:25:00-00:00,2022-07-15,9,19.88617,5,1,1
...,...,...,...,...,...,...,...,...
20635,2022-07-29T22:35:00-00:00,2022-07-29T22:40:00-00:00,2022-07-29,16,100.00000,8,75,0
20636,2022-07-29T22:40:00-00:00,2022-07-29T22:45:00-00:00,2022-07-29,16,104.37120,9,75,0
20637,2022-07-29T22:45:00-00:00,2022-07-29T22:50:00-00:00,2022-07-29,16,118.12000,10,75,0
20638,2022-07-29T22:50:00-00:00,2022-07-29T22:55:00-00:00,2022-07-29,16,121.00000,11,75,0


Try models

In [4]:
def days(d, n):
    '''
    Returns the appropriate number of days to form the train dataset in ahead() function.

    d - number of days you want to include in X_train,
    n - number of the group you want to start from. Should be in [6, 11, 16, 21, 26, 31, 36, 41].
    '''
    if d == 1:
        return [n, n+1, n+3, n+4]                                       # First day

    elif d == 2:
        return [n, n+1, n+3, n+4,                                       # First day
                n+5, n+6, n+8, n+9]                                     # Second day

    elif d == 3:
        return [n, n+1, n+3, n+4,                                       # First day
                n+5, n+6, n+8, n+9,                                     # Second day
                n+10, n+11, n+13, n+14]                                 # Third day

    elif d == 4:
        return [n, n+1, n+3, n+4,                                       # First day
                n+5, n+6, n+8, n+9,                                     # Second day
                n+10, n+11, n+13, n+14,                                 # Third day
                n+15, n+16, n+18, n+19]                                 # Fourth day        

    elif d == 5:
        return [n, n+1, n+3, n+4,                                       # First day
                n+5, n+6, n+8, n+9,                                     # Second day
                n+10, n+11, n+13, n+14,                                 # Third day
                n+15, n+16, n+18, n+19,                                 # Fourth day 
                n+20, n+21, n+23, n+24]                                 # Fifth day

    elif d == 6:
        return [n, n+1, n+3, n+4,                                       # First day
                n+5, n+6, n+8, n+9,                                     # Second day
                n+10, n+11, n+13, n+14,                                 # Third day
                n+15, n+16, n+18, n+19,                                 # Fourth day 
                n+20, n+21, n+23, n+24,                                 # Fifth day
                n+25, n+26, n+28, n+29]                                 # Sixth day    
    
    else: return 'Please enter d in [1:6], or extend the list manually, or ask me (Artem) to do it. I have not made it for d>6 yet'

In [5]:
def ahead(n, d, df, date):
    '''
    Makes train and test datasets and returns MAE.

    n - number of the group you want to start from. Should be in [6, 11, 16, 21, 26, 31, 36, 41],
    d - number of days you want to include in X_train,
    df - preprocessed dataset,
    date - columns to be dropped.
    '''
    train_df = df.loc[df.GROUP.isin(days(d, n))]

    train_df.drop(date, axis=1, inplace=True)
    y_train = train_df.VALUE
    X_train = train_df.drop('VALUE', axis=1)

    n = n+5*d                                               
    test_df = df.loc[df.GROUP.isin([n, n+1, n+3, n+4])]

    test_df.drop(date, axis=1, inplace=True)
    y_test = test_df.VALUE
    X_test = test_df.drop('VALUE', axis=1)

    forest = RandomForestRegressor(n_estimators=20, max_depth=10, criterion='absolute_error')
    forest.fit(X_train, y_train)
    y_pred = forest.predict(X_test)
    return mean_absolute_error(y_test, y_pred)

In [6]:
nn = [6, 11, 16, 21, 26, 31, 36]
res_4 = []
for i in nn:
    res_4.append(ahead(i, 4, df, date))

print('X_train consists of 4 days. The X_train starts to be formed from group == i.\nMAE for each case:', res_4)
print('Mean MAE =', np.mean(res_4))

X_train consists of 4 days. The X_train starts to be formed from group == i.
MAE for each case: [9.365727257595488, 10.983961937282986, 8.571848083116318, 13.709818090060764, 9.343416546657986, 11.585021109157985, 10.23405539171007]
Mean MAE = 10.541978345083084


In [11]:
nn = [6, 11, 16, 21, 26, 31]
res_6 = []
for i in nn:
    res_6.append(ahead(i, 6, df, date))

print('X_train consists of 6 days. The X_train starts to be formed from group == i.\nMAE for each case:', res_6)
print('Mean MAE =', np.mean(res_6))
print('Mean MAE without the possible outlier =', np.mean(res_6[:-1]))
# Something is wrong when we have 6 days in X and start from group=31. I think it's an outlier, but should be checked properly on a broader dataset.

X_train consists of 6 days. The X_train starts to be formed from group == i.
MAE for each case: [7.356427351128472, 13.037939217881943, 10.148298914496527, 10.340709250868056, 9.639224867838541, 33.41474239301216]
Mean MAE = 13.989556999204282
Mean MAE without the possible outlier = 10.104519920442709


In [12]:
# Here you may play with the number of days included in X. As in cells above.
d = 6
ahead(6, d, df, date)

7.47863863780382

### Trash. Old cells.

XGBRegressor

In [118]:
# 4 days
xgbr = xgb.XGBRegressor(n_jobs=-1, random_state=1)
xgbr.fit(X_train, y_train)
y_pred = xgbr.predict(X_test)
mean_absolute_error(y_test, y_pred)

14.61164321677803

DecisionTreeRegressor

In [113]:
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
mean_absolute_error(y_test, y_pred)

11.426156106770833

RandomForestRegressor

In [109]:
forest = RandomForestRegressor(n_estimators=20, max_depth=10, criterion='absolute_error')
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)
mean_absolute_error(y_test, y_pred)

7.367309338975695

### TRASH (function separately)

In [ ]:
n= 6
train_df = df.loc[df.GROUP.isin([n, n+1, n+3, n+4,
n+5, n+6, n+8, n+9,
n+10, n+11, n+13, n+14,
n+15, n+16, n+18, n+19
])]


In [ ]:
train_df.drop(date, axis=1, inplace=True)
y_train = train_df.VALUE

X_train = train_df.drop('VALUE', axis=1)
X_train

,OPR_HR,OPR_INTERVAL,lmp_trans
960,1,1,1
961,1,2,1
962,1,3,1
963,1,4,1
964,1,5,1
...,...,...,...
6715,24,8,0
6716,24,9,0
6717,24,10,0
6718,24,11,0


In [ ]:
n = 26
test_df = df.loc[(df.GROUP == n) | (df.GROUP == n+1) | (df.GROUP == n+3) | (df.GROUP == n+4)]
test_df

,INTERVALSTARTTIME_GMT,INTERVALENDTIME_GMT,OPR_DT,OPR_HR,VALUE,OPR_INTERVAL,GROUP,lmp_trans
6720,2022-07-20T07:00:00-00:00,2022-07-20T07:05:00-00:00,2022-07-20,1,28.24952,1,26,1
6721,2022-07-20T07:05:00-00:00,2022-07-20T07:10:00-00:00,2022-07-20,1,28.25028,2,26,1
6722,2022-07-20T07:10:00-00:00,2022-07-20T07:15:00-00:00,2022-07-20,1,28.55085,3,26,1
6723,2022-07-20T07:15:00-00:00,2022-07-20T07:20:00-00:00,2022-07-20,1,29.18123,4,26,1
6724,2022-07-20T07:20:00-00:00,2022-07-20T07:25:00-00:00,2022-07-20,1,30.27590,5,26,1
...,...,...,...,...,...,...,...,...
8155,2022-07-21T06:35:00-00:00,2022-07-21T06:40:00-00:00,2022-07-20,24,93.32284,8,30,0
8156,2022-07-21T06:40:00-00:00,2022-07-21T06:45:00-00:00,2022-07-20,24,93.23081,9,30,0
8157,2022-07-21T06:45:00-00:00,2022-07-21T06:50:00-00:00,2022-07-20,24,93.50460,10,30,0
8158,2022-07-21T06:50:00-00:00,2022-07-21T06:55:00-00:00,2022-07-20,24,96.36335,11,30,0


In [ ]:
test_df.drop(date, axis=1, inplace=True)
y_test = test_df.VALUE

X_test = test_df.drop('VALUE', axis=1)
X_test

,OPR_HR,OPR_INTERVAL,lmp_trans
6720,1,1,1
6721,1,2,1
6722,1,3,1
6723,1,4,1
6724,1,5,1
...,...,...,...
8155,24,8,0
8156,24,9,0
8157,24,10,0
8158,24,11,0


In [ ]:
forest = RandomForestRegressor(n_estimators=20, max_depth=10, criterion='absolute_error')
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)
mean_absolute_error(y_test, y_pred)

9.171129801432292